In [9]:
from qiskit_gym.envs import LinearFunctionGym, CliffordGym, PermutationGym, LinearFunctionNoisyGym
from qiskit_gym.rl import RLSynthesis, PPOConfig, AlphaZeroConfig, BasicPolicyConfig

from qiskit import QuantumCircuit
from qiskit.transpiler import CouplingMap
import numpy as np

AttributeError: module 'qiskit_gym.qiskit_gym_rs' has no attribute 'LinearFunctionNoisyEnv'

In [20]:
cmap_3_line = CouplingMap.from_line(3, bidirectional=True)
env = LinearFunctionGym.from_coupling_map(cmap_3_line)
env.config["gateset"]

env.difficulty = 1  # We can set env difficulty
env.reset()  # This resets the env and returns first observation

qc = QuantumCircuit(3)
qc.cx(0,2)
qc.draw()

q_0: ──■──
       │  
q_1: ──┼──
     ┌─┴─┐
q_2: ┤ X ├
     └───┘

In [21]:
# You can set a custom state like this (in this case from a circuit)
env.set_state(env.get_state(qc))

env.action_space  # This tells you the number of possible actions (a discrete space of 8 actions)

env.observation_space  # This tells you the type and size of observation space (N by N discrete in this case)

# You can advance the env by providing an action
obs, reward, is_final, _, _ = env.step(2)

# This provides:
# - The observation of the state right after action
# - The reward for that step
# - If we are in a final state

obs, reward, is_final


# One way to do it
env.set_state(env.get_state(qc))
env.render()
for a in [4,2,4]:
    print(f"[{a}] - {env.config['gateset'][a]}")
    obs, reward, is_final, _, _ = env.step(a)
    print(f"[{a}] - Reward: {reward}, Is final: {is_final}")
    env.render()

[[1 0 0]
 [0 1 0]
 [1 0 1]]
[4] - ('SWAP', (0, 1))
[4] - Reward: -0.00390625, Is final: False
[[0 1 0]
 [1 0 0]
 [1 0 1]]
[2] - ('CX', (1, 2))
[2] - Reward: -0.00390625, Is final: False
[[0 1 0]
 [1 0 0]
 [0 0 1]]
[4] - ('SWAP', (0, 1))
[4] - Reward: 1.0, Is final: True
[[1 0 0]
 [0 1 0]
 [0 0 1]]


In [22]:
cmap_3x3 = CouplingMap.from_grid(3,3, bidirectional=False)
env = PermutationGym.from_coupling_map(cmap_3x3)
rls = RLSynthesis(env, PPOConfig(), BasicPolicyConfig())

rls.env.config["gateset"]

[('SWAP', (0, 1)),
 ('SWAP', (0, 3)),
 ('SWAP', (1, 2)),
 ('SWAP', (1, 4)),
 ('SWAP', (2, 5)),
 ('SWAP', (3, 4)),
 ('SWAP', (3, 6)),
 ('SWAP', (4, 5)),
 ('SWAP', (4, 7)),
 ('SWAP', (5, 8)),
 ('SWAP', (6, 7)),
 ('SWAP', (7, 8))]

In [23]:
rls.learn(num_iterations=10, tb_path="runs/perm_square_3x3/")

2025-09-19 09:59:05.237 | INFO     | twisterl.rl.algorithm:learn:176 - (1/0) {'successes': {'ppo_deterministic': 0.0, 'ppo_10': 0.5199999809265137}, 'rewards': {'ppo_deterministic': -0.5078125, 'ppo_10': 0.27421873807907104}, 'difficulty': 1, 'success': 0.0, 'reward': -0.5078125} | {'to_rust': 0.00404725, 'eval_ppo_deterministic': 0.001655167, 'eval_ppo_10': 0.008290458, 'collect': 0.014999666, 'data_to_torch': 0.041117209, 'train': 0.17895525, 'total': 0.249390875}
2025-09-19 09:59:05.244 | INFO     | twisterl.rl.algorithm:learn:196 - (1/0) Improved, saved checkpoint!
2025-09-19 09:59:05.389 | INFO     | twisterl.rl.algorithm:learn:176 - (1/1) {'successes': {'ppo_deterministic': 0.11999999731779099, 'ppo_10': 0.5699999928474426}, 'rewards': {'ppo_deterministic': -0.32734376192092896, 'ppo_10': 0.34941405057907104}, 'difficulty': 1, 'success': 0.11999999731779099, 'reward': -0.32734376192092896} | {'to_rust': 0.003409833, 'eval_ppo_deterministic': 0.001171125, 'eval_ppo_10': 0.00860720

In [24]:
#rls.save("models/perm_square_3x3.json", "models/perm_square_3x3.pt")

rls = RLSynthesis.from_config_json("models/perm_square_3x3.json", "models/perm_square_3x3.pt")

In [25]:
some_perm = np.random.permutation(9)

qc_perm = rls.synth(some_perm, num_searches=1000, num_mcts_searches=0, deterministic=False)
qc_perm.draw(fold=-1)

q_0: ───────────────────X────
                        │    
q_1: ────X──────────────┼────
         │              │    
q_2: ─X──X────────X─────┼────
      │           │     │    
q_3: ─┼───────────┼──X──X──X─
      │           │  │     │ 
q_4: ─┼────────X──┼──X─────X─
      │        │  │          
q_5: ─X─────X──┼──X──────────
            │  │             
q_6: ─X─────┼──┼─────────────
      │     │  │             
q_7: ─X──X──┼──X─────────────
         │  │                
q_8: ────X──X────────────────

In [26]:
qc_perm_input = QuantumCircuit(9)
qc_perm_input.swap(0,8)
qc_perm_input.draw(fold=-1)

q_0: ─X─
      │ 
q_1: ─┼─
      │ 
q_2: ─┼─
      │ 
q_3: ─┼─
      │ 
q_4: ─┼─
      │ 
q_5: ─┼─
      │ 
q_6: ─┼─
      │ 
q_7: ─┼─
      │ 
q_8: ─X─

In [27]:
qc_perm_output = rls.synth(qc_perm_input, num_searches=1000, num_mcts_searches=0, deterministic=False)
qc_perm_output.draw(fold=-1)

q_0: ─X───────────X─
      │           │ 
q_1: ─┼───────────┼─
      │           │ 
q_2: ─┼───────────┼─
      │           │ 
q_3: ─X──X─────X──X─
         │     │    
q_4: ────X──X──X────
            │       
q_5: ─X─────X──X────
      │        │    
q_6: ─┼────────┼────
      │        │    
q_7: ─┼────────┼────
      │        │    
q_8: ─X────────X────

In [28]:
#rls.save("models/lf_5_line.json", "models/lf_5_line.pt")

rls = RLSynthesis.from_config_json("models/lf_5_line.json", "models/lf_5_line.pt")

In [29]:
qc_lf_input = QuantumCircuit(5)
qc_lf_input.cx(0,4)
qc_lf_input.draw(fold=-1)

q_0: ──■──
       │  
q_1: ──┼──
       │  
q_2: ──┼──
       │  
q_3: ──┼──
     ┌─┴─┐
q_4: ┤ X ├
     └───┘

In [30]:
qc_lf_output = rls.synth(qc_lf_input, num_searches=1000, num_mcts_searches=0, deterministic=False)
qc_lf_output.draw(fold=-1)

q_0: ───────■───────────────────■──────────────────────
          ┌─┴─┐               ┌─┴─┐                    
q_1: ──■──┤ X ├──■─────────■──┤ X ├──■─────────────────
     ┌─┴─┐└───┘┌─┴─┐     ┌─┴─┐└───┘┌─┴─┐               
q_2: ┤ X ├─────┤ X ├──■──┤ X ├─────┤ X ├──■────────────
     ├───┤     └───┘┌─┴─┐└───┘     └───┘┌─┴─┐     ┌───┐
q_3: ┤ X ├──■───────┤ X ├───────────────┤ X ├──■──┤ X ├
     └─┬─┘┌─┴─┐     └───┘               └───┘┌─┴─┐└─┬─┘
q_4: ──■──┤ X ├──────────────────────────────┤ X ├──■──
          └───┘                              └───┘

In [31]:
from qiskit.circuit.library.generalized_gates import LinearFunction
LinearFunction(qc_lf_input) == LinearFunction(qc_lf_output)

np.True_

In [36]:
# Here we want to do Clifford synthesis but we only allow H and S to be placed on qubit 0
env = CliffordGym(
    num_qubits=3, 
    gateset=[
        ("CX", [0,1]),
        ("CX", [1,0]),
        ("CX", [1,2]),
        ("CX", [2,1]),
        ("SWAP", [0,1]),
        ("SWAP", [1,2]),
        ("H", [0]),
        ("S", [0]),
    ]
)
rls = RLSynthesis(env, PPOConfig(), BasicPolicyConfig())
rls.learn(num_iterations=10, tb_path="runs/clifford_3q_custom/")

2025-09-19 10:50:25.274 | INFO     | twisterl.rl.algorithm:learn:176 - (1/0) {'successes': {'ppo_deterministic': 0.10999999940395355, 'ppo_10': 0.7300000190734863}, 'rewards': {'ppo_deterministic': -0.3423828184604645, 'ppo_10': 0.590039074420929}, 'difficulty': 1, 'success': 0.10999999940395355, 'reward': -0.3423828184604645} | {'to_rust': 0.00384125, 'eval_ppo_deterministic': 0.001390083, 'eval_ppo_10': 0.006526708, 'collect': 0.010808792, 'data_to_torch': 0.009190958, 'train': 0.109077416, 'total': 0.141090083}
2025-09-19 10:50:25.279 | INFO     | twisterl.rl.algorithm:learn:196 - (1/0) Improved, saved checkpoint!
2025-09-19 10:50:25.421 | INFO     | twisterl.rl.algorithm:learn:176 - (1/1) {'successes': {'ppo_deterministic': 0.14000000059604645, 'ppo_10': 0.6700000166893005}, 'rewards': {'ppo_deterministic': -0.2972656190395355, 'ppo_10': 0.49980467557907104}, 'difficulty': 1, 'success': 0.14000000059604645, 'reward': -0.2972656190395355} | {'to_rust': 0.002984708, 'eval_ppo_determi

In [33]:
#rls.save("models/clifford_3q_custom.json", "models/clifford_3q_custom.pt")

rls = RLSynthesis.from_config_json("models/clifford_3q_custom.json", "models/clifford_3q_custom.pt")

In [40]:
from qiskit.quantum_info import random_clifford, Clifford

In [46]:
qc_clifford_in = QuantumCircuit(3)
qc_clifford_in.h(2)

qc_clifford_out = rls.synth(qc_clifford_in, num_searches=1000, num_mcts_searches=0, deterministic=False)
qc_clifford_out.draw(fold=-1)

┌───┐           
q_0: ────X─┤ H ├─X──────■──
         │ └───┘ │    ┌─┴─┐
q_1: ─X──X───■───X──X─┤ X ├
      │    ┌─┴─┐    │ └───┘
q_2: ─X────┤ X ├────X──────
           └───┘

In [42]:
some_clifford = random_clifford(3, seed=42)
qc_rand_clifford_out = rls.synth(some_clifford, num_searches=1000, num_mcts_searches=0, deterministic=False)
qc_rand_clifford_out.draw(fold=-1)

┌───┐┌───┐   ┌───┐┌───┐┌───┐                        ┌───┐┌───┐          ┌───┐┌───┐                ┌───┐┌───┐┌───┐   ┌───┐┌───┐   ┌───┐   ┌───┐┌───┐┌───┐          ┌───┐┌───┐┌───┐        ┌───┐┌───┐┌───┐┌───┐                ┌───┐┌───┐                                 ┌───┐   ┌───┐        ┌───┐                                 ┌───┐      ┌───┐             ┌───┐┌───┐   ┌───┐                      ┌───┐┌───┐      ┌───┐                     ┌───┐            ┌───┐        
q_0: ┤ S ├┤ H ├───┤ X ├┤ X ├┤ H ├────────────X────────■──┤ X ├┤ H ├──────────┤ X ├┤ H ├──────────────X─┤ X ├┤ S ├┤ H ├─X─┤ H ├┤ H ├─X─┤ H ├─X─┤ H ├┤ H ├┤ X ├──────────┤ X ├┤ S ├┤ S ├──────X─┤ H ├┤ H ├┤ X ├┤ X ├──■───────■─────┤ X ├┤ S ├─────────────────■───X───■───────┤ X ├─X─┤ S ├─X───■──┤ H ├────────────■────■────────────■──┤ H ├──────┤ X ├──────X───■──┤ H ├┤ S ├─X─┤ H ├─X──────■───X──────■──┤ H ├┤ H ├─X────┤ X ├──────X──X────────■──┤ S ├──────────X─┤ H ├────────
     ├───┤└───┘   └─┬─┘└─┬─┘├───┤   ┌───┐    │      ┌─┴─┐└─┬─┘├───┤┌───┐┌───┐└─┬─┘└───┘              │ └─┬─┘└───┘└───┘ │ ├───┤└───┘ │ └───┘ │ └───┘└───┘└─┬─┘┌───┐┌───┐└─┬─┘└───┘└───┘      │ ├───┤└───┘└─┬─┘└─┬─┘┌─┴─┐   ┌─┴─┐   └─┬─┘├───┤               ┌─┴─┐ │ ┌─┴─┐┌───┐└─┬─┘ │ └───┘ │ ┌─┴─┐└───┘┌───┐┌───┐┌─┴─┐┌─┴─┐        ┌─┴─┐└───┘      └─┬─┘      │ ┌─┴─┐└───┘└───┘ │ ├───┤ │    ┌─┴─┐ │    ┌─┴─┐├───┤└───┘ │    └─┬─┘      │  │ ┌───┐┌─┴─┐└───┘          │ └───┘        
q_1: ┤ X ├──X───X───■────■──┤ X ├─X─┤ X ├─X──X───■──┤ X ├──■──┤ X ├┤ X ├┤ X ├──■────■───X───■────■───X───■────■────────X─┤ X ├──────X───────X───X────X────■──┤ X ├┤ X ├──■────■────X────■───X─┤ X ├───────■────■──┤ X ├─X─┤ X ├─X───■──┤ X ├──■────■────■──┤ X ├─X─┤ X ├┤ X ├──■───X───────X─┤ X ├──X──┤ X ├┤ X ├┤ X ├┤ X ├──■───X─┤ X ├──X───X──X───■────■───X─┤ X ├───────────X─┤ X ├─X──X─┤ X ├─X──X─┤ X ├┤ X ├──────X──X───■────■───X──X─┤ X ├┤ X ├──X───X──X──X──X───X────■───X─
     └─┬─┘  │   │           └─┬─┘ │ └─┬─┘ │    ┌─┴─┐└───┘     └─┬─┘└─┬─┘└─┬─┘     ┌─┴─┐ │ ┌─┴─┐┌─┴─┐        ┌─┴─┐        └─┬─┘                  │    │       └─┬─┘└─┬─┘     ┌─┴─┐  │  ┌─┴─┐   └─┬─┘               └───┘ │ └───┘ │      └─┬─┘┌─┴─┐┌─┴─┐┌─┴─┐└───┘   └───┘└─┬─┘                └───┘  │  └─┬─┘└─┬─┘└───┘└───┘┌─┴─┐ │ └───┘  │   │  │      ┌─┴─┐   └───┘             └─┬─┘    │ └───┘    │ └───┘└─┬─┘         │      ┌─┴─┐      └─┬─┘└───┘  │   │  │  │      │  ┌─┴─┐ │ 
q_2: ──■────X───X─────────────■───X───■───X────┤ X ├────────────■────■────■───────┤ X ├─X─┤ X ├┤ X ├────────┤ X ├──────────■────────────────────X────X─────────■────■───────┤ X ├──X──┤ X ├─────■───────────────────────X───────X────────■──┤ X ├┤ X ├┤ X ├───────────────■─────────────────────────X────■────■────────────┤ X ├─X────────X───X──X──────┤ X ├───────────────────────■──────X──────────X────────■───────────X──────┤ X ├────────■─────────X───X──X──X──────X──┤ X ├─X─
                                               └───┘                              └───┘   └───┘└───┘        └───┘                                                           └───┘     └───┘                                                 └───┘└───┘└───┘                                                                └───┘                        └───┘                                                                     └───┘                                      └───┘

In [47]:
# Equivalent up to phase
np.array_equal(some_clifford.tableau[:,:-1], Clifford(qc_rand_clifford_out).tableau[:,:-1])

True